In [1]:
from database import ParserDBHandler

In [2]:
from pathlib import Path

In [4]:
falignment_folder = Path('..').resolve().parent.joinpath('forced_alignment')
db_path = Path('../website/math_corpus_database.db').resolve()

In [5]:
falignment_folder

WindowsPath('C:/Users/Kra$0t04ka/Desktop/me/study/3d_year/workshops/forced_alignment')

In [6]:
class DBHandlerFAlignment(ParserDBHandler):
    
    def get_text_sentences(self, text_name):
        self.cur.execute('''SELECT sents.id, sents.sent
                            FROM sents
                            LEFT JOIN texts
                            ON texts.id = sents.text_id
                            WHERE texts.name = (?)''', (text_name,))
        sentences = self.cur.fetchall()
        return sentences
    
    def add_timecodes(self, timecodes):
        self.cur.executemany('''UPDATE sents
                            SET timecode = (?)
                            WHERE id = (?)''', (timecodes))
        self.conn.commit()

In [63]:
text_name = 'Предел последовательности матан #006 Борис Трушин'

In [64]:
db = DBHandlerFAlignment(db_path)

In [65]:
sentences = db.get_text_sentences(text_name)

In [66]:
sentences[:5]

[(829,
  'Всем привет, с вами снова Борис Трушин, и мы продолжаем заниматься матанализом.'),
 (830,
  'Сегодня поговорим про то, что такое предел последовательности, поговорим о том, как его искать.'),
 (831,
  'Вот, но и дальше будет некоторая серия занятий, где мы обсуждаем пределы последовательности конкретной последовательности, как считать их пределы, но вот, надеюсь, что за эти несколько занятий у вас, даже если были какие-то вопросы, недопонимания, что такое предел последовательность, они так потихонечку снимутся.'),
 (832, 'Для начала определения, да?'),
 (833, 'Что такое предел числовой последовательности?')]

In [67]:
result_folder = falignment_folder.joinpath('_'.join(text_name.split()))
if not result_folder.exists():
    result_folder.mkdir()

In [68]:
txt_filepath = result_folder.joinpath('text.txt')
audio_filepath = result_folder.joinpath('video.mp4')
output_filepath = result_folder.joinpath('alignment.txt')

Создаем текстовый файл, который потом скормим элайнеру

In [69]:
with open(txt_filepath, 'w', encoding='utf-8') as newf:
    newf.write('\n'.join(str(s[0]) + '|' + s[1] for s in sentences))

Создаем команду для элайнера, которую надо будет написать в терминале и запустить

In [70]:
f'python -m aeneas.tools.execute_task {audio_filepath} {txt_filepath} "task_language=eng|is_text_type=parsed|os_task_file_format=tsv" {output_filepath}'

'python -m aeneas.tools.execute_task C:\\Users\\Kra$0t04ka\\Desktop\\me\\study\\3d_year\\workshops\\forced_alignment\\Предел_последовательности_матан_#006_Борис_Трушин\\video.mp4 C:\\Users\\Kra$0t04ka\\Desktop\\me\\study\\3d_year\\workshops\\forced_alignment\\Предел_последовательности_матан_#006_Борис_Трушин\\text.txt "task_language=eng|is_text_type=parsed|os_task_file_format=tsv" C:\\Users\\Kra$0t04ka\\Desktop\\me\\study\\3d_year\\workshops\\forced_alignment\\Предел_последовательности_матан_#006_Борис_Трушин\\alignment.txt'

Теперь запишем таймкоды в базу данных

In [36]:
import csv

In [71]:
timecodes = []
video_start = 0  # секунда по которой обрезали видео
with open(output_filepath, 'r') as f:
    csv_reader = csv.reader(f, delimiter='\t')
    for row in csv_reader:
        timecodes.append((' '.join([str(float(row[0]) + video_start), str(float(row[1]) + video_start)]), int(row[2])))
timecodes[:5]

[('0.0 5.04', 829),
 ('5.04 10.92', 830),
 ('10.92 30.92', 831),
 ('30.92 32.6', 832),
 ('32.6 35.92', 833)]

In [72]:
db.add_timecodes(timecodes)